In [1]:
import numpy as np                               #basically an array
import pandas as pd                              #reading and analyze csv
import matplotlib.pyplot as plt                  #data visualisation
import cv2                                       #comp. vision, image processing, uses numpy as images are 2D array(matrices)
import tensorflow as tf                          #creating neural network (collect, build, train, evaluate, predict)
from PIL import Image, ImageDraw                         #manipulate images in python
import os                                        #directory control
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical           #one-hot encoding
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
import tqdm                                     #progress bar
import warnings
import keras
from google.colab import drive

drive.mount('/content/drive')
directory = "/content/drive/MyDrive/yaren"



Mounted at /content/drive


In [2]:
os.chdir(directory)
print(os.getcwd())

/content/drive/MyDrive/yaren


In [3]:
def inference(path):
    img = keras.utils.load_img(
       path , target_size=(30,30)
    )
    img_array = keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    predictions = model.predict(img_array)
    pred_classes = np.argmax(predictions, axis=1)
    #dictionary to label all traffic signs class.
    classes = { 1:'Speed limit (20km/h)',
                2:'Speed limit (30km/h)',
                3:'Speed limit (50km/h)',
                4:'Speed limit (60km/h)',
                5:'Speed limit (70km/h)',
                6:'Speed limit (80km/h)',
                7:'End of speed limit (80km/h)',
                8:'Speed limit (100km/h)',
                9:'Speed limit (120km/h)',
               10:'No passing',
               11:'No passing veh over 3.5 tons',
               12:'Right-of-way at intersection',
               13:'Priority road',
               14:'Yield',
               15:'Stop',
               16:'No vehicles',
               17:'Veh > 3.5 tons prohibited',
               18:'No entry',
               19:'General caution',
               20:'Dangerous curve left',
               21:'Dangerous curve right',
               22:'Double curve',
               23:'Bumpy road',
               24:'Slippery road',
               25:'Road narrows on the right',
               26:'Road work',
               27:'Traffic signals',
               28:'Pedestrians',
               29:'Children crossing',
               30:'Bicycles crossing',
               31:'Beware of ice/snow',
               32:'Wild animals crossing',
               33:'End speed + passing limits',
               34:'Turn right ahead',
               35:'Turn left ahead',
               36:'Ahead only',
               37:'Go straight or right',
               38:'Go straight or left',
               39:'Keep right',
               40:'Keep left',
               41:'Roundabout mandatory',
               42:'End of no passing',
               43:'End no passing veh > 3.5 tons' }


    print("Class = ",classes[pred_classes[0]]," ||   acc = ",max(predictions[0]))







In [4]:
data = []
labels = []
classes = 43

for i in range(classes):
    path = os.path.join(os.getcwd(),'Train',str(i))
    images = os.listdir(path)

    for j in images:
        try:
            image = Image.open(path +'/'+j)
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")


#Converting lists into numpy arrays bcoz its faster and takes lesser memory
data = np.array(data)
labels = np.array(labels)

print(data.shape, labels.shape)
# Total Images:39209 of size (30*30*3(3 Means Color))
data[39000] #peaking
labels[4900]
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=68)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
y_train[20000]

(39219, 30, 30, 3) (39219,)
(31375, 30, 30, 3) (7844, 30, 30, 3) (31375,) (7844,)
(31375, 30, 30, 3) (7844, 30, 30, 3) (31375, 43) (7844, 43)


array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [6]:
#Building the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

warnings.filterwarnings("ignore", category=DeprecationWarning)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train, batch_size=32, epochs=6, validation_data=(X_test, y_test))
#Final trainig of model

warnings.filterwarnings("ignore", category=DeprecationWarning)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        2432      
                                                                 
 conv2d_5 (Conv2D)           (None, 22, 22, 32)        25632     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 11, 11, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 11, 11, 32)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 9, 9, 64)          18496     
                                                                 
 conv2d_7 (Conv2D)           (None, 7, 7, 64)          36928     
                                                      

In [7]:
model.save("Trafic_signs_model3.h5")

In [8]:
pred_path = os.path.join('Test',"12571.png")
inference(pred_path)

1/1 [==============================] - 1s 544ms/step
Class =  Speed limit (30km/h)  ||   acc =  0.98403686
